In [1]:
import sys
sys.path.append('../src/')

%load_ext autoreload
%autoreload 2
# Importing our custom module(s)
import losses
import utils

In [2]:
import os
import itertools
import re
import numpy as np
import pandas as pd
# PyTorch
import torch

In [3]:
'''Code to view category prevalence'''
dataset_path = '/cluster/tufts/hugheslab/eharve06/HAM10000'
df = pd.read_csv(f'{dataset_path}/HAM10000_metadata')
df.dx.value_counts()/df.shape[0]

nv       0.669496
mel      0.111133
bkl      0.109735
bcc      0.051323
akiec    0.032651
vasc     0.014179
df       0.011483
Name: dx, dtype: float64

In [4]:
'''Code to create labels.csv'''
def label_to_category(label, categories):
    if label not in categories: return [0]
    else: return [categories.index(label)+1]

dataset_path = '/cluster/tufts/hugheslab/eharve06/HAM10000'
df = pd.read_csv(f'{dataset_path}/HAM10000_metadata')
categories = ['bkl', 'nv', 'df', 'mel', 'vasc', 'bcc', 'akiec']
categories = ['bkl', 'nv', 'mel'] # categories based off prevalence
df['label'] = df['dx'].apply(lambda label: label_to_category(label, categories))
df['path'] = df['image_id'].apply(lambda image_id: f'{dataset_path}/train/{image_id}.jpg')
temp_df = df[['lesion_id', 'label', 'path']].set_index('lesion_id')
temp_df.to_csv(f'{dataset_path}/train/labels.csv')
temp_df.head()

,label,path
lesion_id,,
HAM_0000118,[1],/cluster/tufts/hugheslab/eharve06/HAM10000/tra...
HAM_0000118,[1],/cluster/tufts/hugheslab/eharve06/HAM10000/tra...
HAM_0002730,[1],/cluster/tufts/hugheslab/eharve06/HAM10000/tra...
HAM_0002730,[1],/cluster/tufts/hugheslab/eharve06/HAM10000/tra...
HAM_0001466,[1],/cluster/tufts/hugheslab/eharve06/HAM10000/tra...


In [5]:
dataset_path = '/cluster/tufts/hugheslab/eharve06/HAM10000'
# TODO: Remove ISIC_0035068.jpg from ISIC2018_Task3_Test_GroundTruth.csv,
#       the image is missing from ISIC2018_Task3_Test_Images.zip
df = pd.read_csv(f'{dataset_path}/ISIC2018_Task3_Test_GroundTruth.csv')
categories = ['bkl', 'nv', 'df', 'mel', 'vasc', 'bcc', 'akiec']
categories = ['bkl', 'nv', 'mel'] # categories based off prevalence
df['label'] = df['dx'].apply(lambda label: label_to_category(label, categories))
df['path'] = df['image_id'].apply(lambda image_id: f'{dataset_path}/test/{image_id}.jpg')
temp_df = df[['lesion_id', 'label', 'path']].set_index('lesion_id')
temp_df.to_csv(f'{dataset_path}/test/labels.csv')
temp_df.head()

,label,path
lesion_id,,
HAMTEST_0000000,[2],/cluster/tufts/hugheslab/eharve06/HAM10000/tes...
HAMTEST_0000001,[2],/cluster/tufts/hugheslab/eharve06/HAM10000/tes...
HAMTEST_0000002,[1],/cluster/tufts/hugheslab/eharve06/HAM10000/tes...
HAMTEST_0000003,[2],/cluster/tufts/hugheslab/eharve06/HAM10000/tes...
HAMTEST_0000004,[2],/cluster/tufts/hugheslab/eharve06/HAM10000/tes...


In [6]:
dataset_path = '/cluster/tufts/hugheslab/eharve06/HAM10000'
experiments_path = '/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tuned_HAM10000'
lr_0s = [0.1, 0.01, 0.001, 0.0001]
ns = [100, 1000]
prior_path = '/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior'
prior_type = 'nonlearned'
random_states = [1001, 2001, 3001]
wandb_project = 'tuned_HAM10000'
weight_decays = [0.01, 0.001, 0.0001, 1e-05, 1e-06, 0.0]
count = -1
for lr_0, n, random_state, weight_decay in itertools.product(lr_0s, ns, random_states, weight_decays):
    count += 1
    model_name = f'{prior_type}_lr_0={lr_0}_n={n}_random_state={random_state}_weight_decay={weight_decay}'
    print(f'    "python ../src/main_HAM10000.py --bb_weight_decay={weight_decay} --clf_weight_decay={weight_decay} --dataset_path=\'{dataset_path}\' --experiments_path=\'{experiments_path}\' --lr_0={lr_0} --n={n} --model_name=\'{model_name}\' --prior_path=\'{prior_path}\' --random_state={random_state} --tune --wandb --wandb_project=\'{wandb_project}\'"')
print(count)

    "python ../src/main_HAM10000.py --bb_weight_decay=0.01 --clf_weight_decay=0.01 --dataset_path='/cluster/tufts/hugheslab/eharve06/HAM10000' --experiments_path='/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tuned_HAM10000' --lr_0=0.1 --n=100 --model_name='nonlearned_lr_0=0.1_n=100_random_state=1001_weight_decay=0.01' --prior_path='/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior' --random_state=1001 --tune --wandb --wandb_project='tuned_HAM10000'"
    "python ../src/main_HAM10000.py --bb_weight_decay=0.001 --clf_weight_decay=0.001 --dataset_path='/cluster/tufts/hugheslab/eharve06/HAM10000' --experiments_path='/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tuned_HAM10000' --lr_0=0.1 --n=100 --model_name='nonlearned_lr_0=0.1_n=100_random_state=1001_weight_decay=0.001' --prior_path='/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior' --random_state=1001 --tune --wandb --wandb_project='tuned_HAM10000'"
    "python ../src/main_HAM10000.py -

In [7]:
dataset_path = '/cluster/tufts/hugheslab/eharve06/HAM10000'
experiments_path = '/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tuned_HAM10000'
lr_0s = [0.1, 0.01, 0.001, 0.0001]
ns = [100, 1000]
prior_path = '/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior'
prior_type = 'adapted'
random_states = [1001, 2001, 3001]
wandb_project = 'tuned_HAM10000'
weight_decays = [0.01, 0.001, 0.0001, 1e-05, 1e-06, 0.0]
count = -1
for lr_0, n, random_state, weight_decay in itertools.product(lr_0s, ns, random_states, weight_decays):
    count += 1
    model_name = f'{prior_type}_lr_0={lr_0}_n={n}_random_state={random_state}_weight_decay={weight_decay}'
    print(f'    "python ../src/main_HAM10000.py --bb_weight_decay={weight_decay} --clf_weight_decay={weight_decay} --dataset_path=\'{dataset_path}\' --experiments_path=\'{experiments_path}\' --lr_0={lr_0} --n={n} --model_name=\'{model_name}\' --prior_path=\'{prior_path}\' --prior_type=\'{prior_type}\' --random_state={random_state} --tune --wandb --wandb_project=\'{wandb_project}\'"')
print(count)

    "python ../src/main_HAM10000.py --bb_weight_decay=0.01 --clf_weight_decay=0.01 --dataset_path='/cluster/tufts/hugheslab/eharve06/HAM10000' --experiments_path='/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tuned_HAM10000' --lr_0=0.1 --n=100 --model_name='adapted_lr_0=0.1_n=100_random_state=1001_weight_decay=0.01' --prior_path='/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior' --prior_type='adapted' --random_state=1001 --tune --wandb --wandb_project='tuned_HAM10000'"
    "python ../src/main_HAM10000.py --bb_weight_decay=0.001 --clf_weight_decay=0.001 --dataset_path='/cluster/tufts/hugheslab/eharve06/HAM10000' --experiments_path='/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tuned_HAM10000' --lr_0=0.1 --n=100 --model_name='adapted_lr_0=0.1_n=100_random_state=1001_weight_decay=0.001' --prior_path='/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior' --prior_type='adapted' --random_state=1001 --tune --wandb --wandb_project='tuned_HAM10000

In [8]:
bb_weight_decays = np.logspace(0, 9, num=10)
clf_weight_decays = [0.01, 0.001, 0.0001, 1e-05, 1e-06, 0.0]
dataset_path = '/cluster/tufts/hugheslab/eharve06/HAM10000'
experiments_path = '/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tuned_HAM10000'
lr_0s = [0.1, 0.01, 0.001, 0.0001]
ns = [100, 1000]
prior_path = '/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior'
prior_type = 'LearnedPriorLR'
random_states = [1001, 2001, 3001]
wandb_project = 'tuned_HAM10000'
count = -1
for bb_weight_decay, clf_weight_decay, lr_0, n, random_state in itertools.product(bb_weight_decays, clf_weight_decays, lr_0s, ns, random_states):
    count += 1
    model_name = f'{prior_type}_bb_weight_decay={bb_weight_decay}_clf_weight_decay={clf_weight_decay}_lr_0={lr_0}_n={n}_random_state={random_state}'
    print(f'    "python ../src/main_HAM10000.py --bb_weight_decay={bb_weight_decay} --clf_weight_decay={clf_weight_decay} --dataset_path=\'{dataset_path}\' --experiments_path=\'{experiments_path}\' --lr_0={lr_0} --n={n} --model_name=\'{model_name}\' --prior_path=\'{prior_path}\' --prior_type=\'{prior_type}\' --random_state={random_state} --tune --wandb --wandb_project=\'{wandb_project}\'"')
print(count)

    "python ../src/main_HAM10000.py --bb_weight_decay=1.0 --clf_weight_decay=0.01 --dataset_path='/cluster/tufts/hugheslab/eharve06/HAM10000' --experiments_path='/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tuned_HAM10000' --lr_0=0.1 --n=100 --model_name='LearnedPriorLR_bb_weight_decay=1.0_clf_weight_decay=0.01_lr_0=0.1_n=100_random_state=1001' --prior_path='/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior' --prior_type='LearnedPriorLR' --random_state=1001 --tune --wandb --wandb_project='tuned_HAM10000'"
    "python ../src/main_HAM10000.py --bb_weight_decay=1.0 --clf_weight_decay=0.01 --dataset_path='/cluster/tufts/hugheslab/eharve06/HAM10000' --experiments_path='/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tuned_HAM10000' --lr_0=0.1 --n=100 --model_name='LearnedPriorLR_bb_weight_decay=1.0_clf_weight_decay=0.01_lr_0=0.1_n=100_random_state=2001' --prior_path='/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior' --prior_type='LearnedPrio

In [9]:
def get_df(path):
    df = pd.read_csv(path, index_col='Unnamed: 0')
    return df

def get_val_nll(df):
    return df.val_or_test_nll.values[-1]

def get_val_acc1(df):
    if 'val_or_test_acc' in df.columns:
        return df.val_or_test_acc.values[-1]
    elif 'val_or_test_acc1' in df.columns:
        return df.val_or_test_acc1.values[-1]

In [10]:
def get_best_hyperparams(
    bb_weight_decays, 
    clf_weight_decays, 
    experiments_path, # Gets best hyperparameters from experiments_path
    lr_0s, 
    ns, 
    prior_type, # Gets best hyperparameters for a single prior_type
    random_states, 
    weight_decays
):
    hyperparam_names = ['bb_weight_decay', 'clf_weight_decay', 'lr_0', 'n', 'random_state', 'weight_decay']
    columns = ['model_name', 'n', 'prior_type', 'random_state', 'val_acc1', 'val_nll']
    df = pd.DataFrame(columns=columns)
    
    for hyperparams in itertools.product(bb_weight_decays, clf_weight_decays, lr_0s, ns, random_states, weight_decays):
        model_name = f'{prior_type}_' + '_'.join(f'{hyperparam_name}={hyperparam}' for hyperparam_name, hyperparam in zip(hyperparam_names, hyperparams) if hyperparam is not None)
        bb_weight_decay, clf_weight_decay, lr_0, n, random_state, weight_decay = hyperparams
        if not os.path.exists(f'{experiments_path}/{model_name}.csv'):
            print(model_name)
            pass
        else:
            temp_df = get_df(f'{experiments_path}/{model_name}.csv')
            val_acc1 = get_val_acc1(temp_df)
            val_nll = get_val_nll(temp_df)
            df.loc[df.shape[0]] = [model_name, n, prior_type, random_state, val_acc1, val_nll]
                    
    min_indices = df.groupby(['n', 'random_state'])['val_nll'].idxmin()
    df = df.loc[min_indices].reset_index(drop=True)
    #max_indices = df.groupby(['n', 'random_state'])['val_acc1'].idxmax()
    #df = df.loc[max_indices].reset_index(drop=True)
                    
    return df

def get_results(
    df,
    experiments_path
):
    columns = ['model_name', 'n', 'prior_type', 'random_state', 'test_acc1', 'test_nll']
    results_df = pd.DataFrame(columns=columns)
    
    for row_index, row in df.iterrows():
        model_name = row.model_name
        prior_type, n, random_state = row.prior_type, row.n, row.random_state
        if not os.path.exists(f'{experiments_path}/{model_name}.csv'):
            #print(model_name)
            pass
        else:
            temp_df = get_df(f'{experiments_path}/{model_name}.csv')
            test_acc1 = get_val_acc1(temp_df)
            test_nll = get_val_nll(temp_df)
            condition = (results_df['n']==n)&(results_df['random_state']==random_state)
            if len(results_df[condition]) == 0:
                results_df.loc[results_df.shape[0]] = [model_name, n, prior_type, random_state, test_acc1, test_nll]
    
    return results_df

In [11]:
dataset_path = '/cluster/tufts/hugheslab/eharve06/HAM10000'
experiments_path = '/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tuned_HAM10000'
lr_0s = [0.1, 0.01, 0.001, 0.0001]
ns = [100, 1000]
prior_path = '/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior'
prior_type = 'nonlearned'
random_states = [1001, 2001, 3001]
wandb_project = 'tuned_HAM10000'
weight_decays = [0.01, 0.001, 0.0001, 1e-05, 1e-06, 0.0]

std_prior = get_best_hyperparams([None], [None], experiments_path, lr_0s, ns, prior_type, random_states, weight_decays)
std_prior

nonlearned_lr_0=0.1_n=100_random_state=1001_weight_decay=0.01
nonlearned_lr_0=0.1_n=100_random_state=1001_weight_decay=0.001
nonlearned_lr_0=0.1_n=100_random_state=1001_weight_decay=0.0001
nonlearned_lr_0=0.1_n=100_random_state=1001_weight_decay=1e-05
nonlearned_lr_0=0.1_n=100_random_state=1001_weight_decay=1e-06
nonlearned_lr_0=0.1_n=100_random_state=1001_weight_decay=0.0
nonlearned_lr_0=0.1_n=100_random_state=2001_weight_decay=0.01
nonlearned_lr_0=0.1_n=100_random_state=2001_weight_decay=0.001
nonlearned_lr_0=0.1_n=100_random_state=2001_weight_decay=0.0001
nonlearned_lr_0=0.1_n=100_random_state=2001_weight_decay=1e-05
nonlearned_lr_0=0.1_n=100_random_state=2001_weight_decay=1e-06
nonlearned_lr_0=0.1_n=100_random_state=2001_weight_decay=0.0
nonlearned_lr_0=0.1_n=100_random_state=3001_weight_decay=0.01
nonlearned_lr_0=0.1_n=100_random_state=3001_weight_decay=0.001
nonlearned_lr_0=0.1_n=100_random_state=3001_weight_decay=0.0001
nonlearned_lr_0=0.1_n=100_random_state=3001_weight_decay=1e

,model_name,n,prior_type,random_state,val_acc1,val_nll


In [12]:
dataset_path = '/cluster/tufts/hugheslab/eharve06/HAM10000'
experiments_path = '/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/retrained_HAM10000'
prior_path = '/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior'
wandb_project = 'retrained_HAM10000'

pattern = re.compile(r'(\w+)_lr_0=([\d.]+)_n=(\d+)_random_state=(\d+)_weight_decay=([\d.]+(?:e[-+]?\d+)?)')
for row_index, row in std_prior.iterrows():
    model_name = row.model_name
    match = pattern.match(model_name)
    prior_type, lr_0, n, random_state, weight_decay = match.groups()
    if not os.path.exists(f'{experiments_path}/{model_name}.csv'):
        print(f'    "python ../src/main_HAM10000.py --bb_weight_decay={weight_decay} --clf_weight_decay={weight_decay} --dataset_path=\'{dataset_path}\' --experiments_path=\'{experiments_path}\' --lr_0={lr_0} --n={n} --model_name=\'{model_name}\' --prior_path=\'{prior_path}\' --prior_type=\'{prior_type}\' --random_state={random_state} --save --wandb --wandb_project=\'{wandb_project}\'"')

In [13]:
dataset_path = '/cluster/tufts/hugheslab/eharve06/HAM10000'
experiments_path = '/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tuned_HAM10000'
lr_0s = [0.1, 0.01, 0.001, 0.0001]
ns = [100, 1000]
prior_path = '/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior'
prior_type = 'adapted'
random_states = [1001, 2001, 3001]
wandb_project = 'tuned_HAM10000'
weight_decays = [0.01, 0.001, 0.0001, 1e-05, 1e-06, 0.0]

learned_prior_iso = get_best_hyperparams([None], [None], experiments_path, lr_0s, ns, prior_type, random_states, weight_decays)
learned_prior_iso

adapted_lr_0=0.1_n=100_random_state=1001_weight_decay=0.01
adapted_lr_0=0.1_n=100_random_state=1001_weight_decay=0.001
adapted_lr_0=0.1_n=100_random_state=1001_weight_decay=0.0001
adapted_lr_0=0.1_n=100_random_state=1001_weight_decay=1e-05
adapted_lr_0=0.1_n=100_random_state=1001_weight_decay=1e-06
adapted_lr_0=0.1_n=100_random_state=1001_weight_decay=0.0
adapted_lr_0=0.1_n=100_random_state=2001_weight_decay=0.01
adapted_lr_0=0.1_n=100_random_state=2001_weight_decay=0.001
adapted_lr_0=0.1_n=100_random_state=2001_weight_decay=0.0001
adapted_lr_0=0.1_n=100_random_state=2001_weight_decay=1e-05
adapted_lr_0=0.1_n=100_random_state=2001_weight_decay=1e-06
adapted_lr_0=0.1_n=100_random_state=2001_weight_decay=0.0
adapted_lr_0=0.1_n=100_random_state=3001_weight_decay=0.01
adapted_lr_0=0.1_n=100_random_state=3001_weight_decay=0.001
adapted_lr_0=0.1_n=100_random_state=3001_weight_decay=0.0001
adapted_lr_0=0.1_n=100_random_state=3001_weight_decay=1e-05
adapted_lr_0=0.1_n=100_random_state=3001_wei

,model_name,n,prior_type,random_state,val_acc1,val_nll


In [14]:
dataset_path = '/cluster/tufts/hugheslab/eharve06/HAM10000'
experiments_path = '/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/retrained_HAM10000'
prior_path = '/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior'
wandb_project = 'retrained_HAM10000'

pattern = re.compile(r'(\w+)_lr_0=([\d.]+)_n=(\d+)_random_state=(\d+)_weight_decay=([\d.]+(?:e[-+]?\d+)?)')
for row_index, row in learned_prior_iso.iterrows():
    model_name = row.model_name
    match = pattern.match(model_name)
    prior_type, lr_0, n, random_state, weight_decay = match.groups()
    if not os.path.exists(f'{experiments_path}/{model_name}.csv'):
        print(f'    "python ../src/main_HAM10000.py --bb_weight_decay={weight_decay} --clf_weight_decay={weight_decay} --dataset_path=\'{dataset_path}\' --experiments_path=\'{experiments_path}\' --lr_0={lr_0} --n={n} --model_name=\'{model_name}\' --prior_path=\'{prior_path}\' --prior_type=\'{prior_type}\' --random_state={random_state} --save --wandb --wandb_project=\'{wandb_project}\'"')

In [15]:
bb_weight_decays = np.logspace(0, 9, num=10)
clf_weight_decays = [0.01, 0.001, 0.0001, 1e-05, 1e-06, 0.0]
dataset_path = '/cluster/tufts/hugheslab/eharve06/HAM10000'
experiments_path = '/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tuned_HAM10000'
lr_0s = [0.1, 0.01, 0.001, 0.0001]
ns = [100, 1000]
prior_path = '/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior'
prior_type = 'LearnedPriorLR'
random_states = [1001, 2001, 3001]
wandb_project = 'tuned_HAM10000'

learned_prior_lr = get_best_hyperparams(bb_weight_decays, clf_weight_decays, experiments_path, lr_0s, ns, prior_type, random_states, [None])
learned_prior_lr

,model_name,n,prior_type,random_state,val_acc1,val_nll
0,LearnedPriorLR_bb_weight_decay=1000000.0_clf_w...,100,LearnedPriorLR,1001,0.619420,0.658586
1,LearnedPriorLR_bb_weight_decay=100.0_clf_weigh...,100,LearnedPriorLR,2001,0.455357,0.341032
2,LearnedPriorLR_bb_weight_decay=10.0_clf_weight...,100,LearnedPriorLR,3001,0.820238,0.832796
3,LearnedPriorLR_bb_weight_decay=1000.0_clf_weig...,1000,LearnedPriorLR,1001,0.867026,0.735219
4,LearnedPriorLR_bb_weight_decay=1000.0_clf_weig...,1000,LearnedPriorLR,2001,0.895292,0.768048
5,LearnedPriorLR_bb_weight_decay=1.0_clf_weight_...,1000,LearnedPriorLR,3001,0.857513,0.776166


In [16]:
dataset_path = '/cluster/tufts/hugheslab/eharve06/HAM10000'
experiments_path = '/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/retrained_HAM10000'
prior_path = '/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior'
wandb_project = 'retrained_HAM10000'

pattern = re.compile(r'(\w+)_bb_weight_decay=([\d.]+)_clf_weight_decay=([\d.]+(?:e[-+]?\d+)?)_lr_0=([\d.]+)_n=(\d+)_random_state=(\d+)')
for row_index, row in learned_prior_lr.iterrows():
    model_name = row.model_name
    match = pattern.match(model_name)
    prior_type, bb_weight_decay, clf_weight_decay, lr_0, n, random_state = match.groups()
    if not os.path.exists(f'{experiments_path}/{model_name}.csv'):
        print(f'    "python ../src/main_HAM10000.py --bb_weight_decay={bb_weight_decay} --clf_weight_decay={clf_weight_decay} --dataset_path=\'{dataset_path}\' --experiments_path=\'{experiments_path}\' --lr_0={lr_0} --n={n} --model_name=\'{model_name}\' --prior_path=\'{prior_path}\' --prior_type=\'{prior_type}\' --random_state={random_state} --save --wandb --wandb_project=\'{wandb_project}\'"')

In [17]:
experiments_path = '/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/retrained_HAM10000'
std_prior_results = get_results(std_prior, experiments_path)
learned_prior_iso_results = get_results(learned_prior_iso, experiments_path)
learned_prior_lr_results = get_results(learned_prior_lr, experiments_path)
results = pd.concat([std_prior_results, learned_prior_iso_results, learned_prior_lr_results])
results

,model_name,n,prior_type,random_state,test_acc1,test_nll
0,LearnedPriorLR_bb_weight_decay=1000000.0_clf_w...,100,LearnedPriorLR,1001,0.748520,1.272245
1,LearnedPriorLR_bb_weight_decay=100.0_clf_weigh...,100,LearnedPriorLR,2001,0.786883,1.897069
2,LearnedPriorLR_bb_weight_decay=10.0_clf_weight...,100,LearnedPriorLR,3001,0.826547,0.892638
3,LearnedPriorLR_bb_weight_decay=1000.0_clf_weig...,1000,LearnedPriorLR,1001,0.849875,0.801882
4,LearnedPriorLR_bb_weight_decay=1000.0_clf_weig...,1000,LearnedPriorLR,2001,0.874180,1.093244
5,LearnedPriorLR_bb_weight_decay=1.0_clf_weight_...,1000,LearnedPriorLR,3001,0.874762,0.783335


In [18]:
grouped_results = results.groupby(['n', 'prior_type']).agg(lambda x: tuple(x))
columns = ['test_acc1', 'test_nll']
for column in columns:
    grouped_results[f'{column}_mean'] = grouped_results[column].apply(lambda item: np.mean(item))
    grouped_results[f'{column}_std'] = grouped_results[column].apply(lambda item: np.std(item))
    grouped_results[f'{column}_min'] = grouped_results[column].apply(lambda item: np.min(item))
    grouped_results[f'{column}_max'] = grouped_results[column].apply(lambda item: np.max(item))
grouped_results = grouped_results.reset_index()
grouped_results

,n,prior_type,model_name,random_state,test_acc1,test_nll,test_acc1_mean,test_acc1_std,test_acc1_min,test_acc1_max,test_nll_mean,test_nll_std,test_nll_min,test_nll_max
0,100,LearnedPriorLR,(LearnedPriorLR_bb_weight_decay=1000000.0_clf_...,"(1001, 2001, 3001)","(0.7485202550888062, 0.786882758140564, 0.8265...","(1.2722452530239214, 1.897069216090113, 0.8926...",0.787317,0.031856,0.748520,0.826547,1.353984,0.414111,0.892638,1.897069
1,1000,LearnedPriorLR,(LearnedPriorLR_bb_weight_decay=1000.0_clf_wei...,"(1001, 2001, 3001)","(0.849875271320343, 0.8741799592971802, 0.8747...","(0.8018823213564648, 1.0932437508884127, 0.783...",0.866272,0.011597,0.849875,0.874762,0.892820,0.141923,0.783335,1.093244


In [19]:
grouped_results[['n', 'prior_type', 'test_acc1_mean', 'test_acc1_min', 'test_acc1_max']]

,n,prior_type,test_acc1_mean,test_acc1_min,test_acc1_max
0,100,LearnedPriorLR,0.787317,0.748520,0.826547
1,1000,LearnedPriorLR,0.866272,0.849875,0.874762


In [20]:
grouped_results[['n', 'prior_type', 'test_nll_mean', 'test_nll_min', 'test_nll_max']]

,n,prior_type,test_nll_mean,test_nll_min,test_nll_max
0,100,LearnedPriorLR,1.353984,0.892638,1.897069
1,1000,LearnedPriorLR,0.892820,0.783335,1.093244
